# Image preparation

In [3]:
import glob, os
from PIL import Image
from resizeimage import resizeimage
import numpy
import pywt
import sys
import re
from random import shuffle

FOLDER_LOG = "log"
FOLDER_IMAGES = "images"
FOLDER_IMAGES_SOURCE = "images/source"
PROCESSED_IMAGES_FOLDER = "images/processed"
FOLDER_GRAPH = "graph"

PROCESSED_IMAGES_FILE = os.path.join(FOLDER_LOG, "processed_images.txt")
SKIPPED_IMAGES_FILE = os.path.join(FOLDER_LOG, "skipped_images.txt")

PERFORM_BLUR_ANALYSIS = False
#settings for blurred images processing
thresh = 35
MinZero = 0.05

#get source images folders (skip hidden)
folders_to_check = [folder for folder in os.listdir(FOLDER_IMAGES_SOURCE) if folder[0] != '.'] 

if not os.path.exists(FOLDER_LOG):
        os.makedirs(FOLDER_LOG)

skipped_images_count = 0
unprocessed_images = []
processed_images = []
skipped_images = []
if os.path.exists(PROCESSED_IMAGES_FILE):
    with open(PROCESSED_IMAGES_FILE) as f:
        processed_images = [line.replace("\n", "") for line in f.readlines()]

if os.path.exists(SKIPPED_IMAGES_FILE):
    with open(SKIPPED_IMAGES_FILE) as f:
        skipped_images = [line.replace("\n", "") for line in f.readlines()]
        
processed_images += skipped_images
    
for folder in folders_to_check:
    for f in os.listdir(os.path.join(FOLDER_IMAGES_SOURCE, folder)):
        if os.path.splitext(f)[1].lower() in ('.jpg', '.jpeg'):
            file_path = os.path.join(folder, f)
            if not file_path in processed_images:
                unprocessed_images.append(os.path.join(FOLDER_IMAGES_SOURCE, file_path))
    target_folder = os.path.join(PROCESSED_IMAGES_FOLDER, folder)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
print("Number of previously processed images: %d" % len(processed_images))
print("Number of images to process: %d" % len(unprocessed_images))

#shuffle list
shuffle(unprocessed_images)

unprocessed_images_count = 0
processed_images_count = 0
for image_path in unprocessed_images:
    img=Image.open(image_path)
    max_dim = img.size.index(max(img.size))
    if max(img.size) < 320:
        continue
    try:
        if max_dim == 0:
            img = resizeimage.resize_width(img, 320)
        else:
            img = resizeimage.resize_height(img, 320)
    except:
        continue
    """ This is an implementation of the paper found here: http://www.cs.cmu.edu/~htong/pdf/ICME04_tong.pdf """
    if PERFORM_BLUR_ANALYSIS:
        im = img.convert('F')
        x=numpy.asarray(im)
        x_cropped=x[0:(numpy.shape(x)[0]/16)*16 - 1,0:(numpy.shape(x)[1]/16)*16 - 1]
        LL1,(LH1,HL1,HH1)=pywt.dwt2(x_cropped,'haar')
        LL2,(LH2,HL2,HH2)=pywt.dwt2(LL1      ,'haar')
        LL3,(LH3,HL3,HH3)=pywt.dwt2(LL2      ,'haar')
        Emap1=numpy.square(LH1) + numpy.square(HL1) + numpy.square(HH1)
        Emap2=numpy.square(LH2) + numpy.square(HL2) + numpy.square(HH2)
        Emap3=numpy.square(LH3) + numpy.square(HL3) + numpy.square(HH3)

        dimx=numpy.shape(Emap1)[0]/8
        dimy=numpy.shape(Emap1)[1]/8
        Emax1=[]
        vert=1
        for j in range(0,dimx - 2):
            horz=1;
            Emax1.append([])
            for k in range(0,dimy - 2):
                Emax1[j].append(numpy.max(numpy.max(Emap1[vert:vert+7,horz:horz+7])))
                horz=horz+8
            vert=vert+8

        dimx=numpy.shape(Emap2)[0]/4
        dimy=numpy.shape(Emap2)[1]/4
        Emax2=[]
        vert=1
        for j in range(0,dimx - 2):
            horz=1;
            Emax2.append([])
            for k in range(0,dimy - 2):
                Emax2[j].append(numpy.max(numpy.max(Emap2[vert:vert+3,horz:horz+3])))
                horz=horz+4
            vert=vert+4

        dimx=numpy.shape(Emap3)[0]/2
        dimy=numpy.shape(Emap3)[1]/2
        Emax3=[]
        vert=1
        for j in range(0,dimx - 2):
            horz=1;
            Emax3.append([])
            for k in range(0,dimy - 2):
                Emax3[j].append(numpy.max(numpy.max(Emap3[vert:vert+1,horz:horz+1])))
                horz=horz+2
            vert=vert+2

        N_edge=0
        N_da=0
        N_rg=0
        N_brg=0

        EdgeMap = []
        for j in range(0, dimx - 2):
            EdgeMap.append([])
            for k in range(0, dimy - 2):
                if (Emax1[j][k]>thresh) or (Emax2[j][k]>thresh) or (Emax3[j][k]>thresh):
                    EdgeMap[j].append(1)
                    N_edge = N_edge + 1
                    rg = 0
                    if (Emax1[j][k]>Emax2[j][k]) and (Emax2[j][k]>Emax3[j][k]):
                        N_da=N_da+1
                    elif (Emax1[j][k]<Emax2[j][k]) and (Emax2[j][k]<Emax3[j][k]):
                        rg = 1
                        N_rg=N_rg+1
                    elif (Emax2[j][k]>Emax1[j][k]) and (Emax2[j][k]>Emax3[j][k]):
                        rg = 1
                        N_rg=N_rg+1
                    if rg and (Emax1[j][k]<thresh):
                        N_brg=N_brg+1
                else:
                    EdgeMap[j].append(0)

        per=float(N_da)/N_edge
        BlurExtent=float(N_brg)/N_rg

    else:
        BlurExtent = 0
    if BlurExtent < 0.5:
        subfolder_path_indices = [m.start() for m in re.finditer('/', image_path)]
        if len(subfolder_path_indices) > 1:
            subfolder_path_index = subfolder_path_indices[-2]
        else:
            subfolder_path_index = 0
            
        target_path = PROCESSED_IMAGES_FOLDER + image_path[subfolder_path_index:]
        img.save(target_path , img.format)
        with open(PROCESSED_IMAGES_FILE, "a") as myfile:
            myfile.write(image_path[subfolder_path_index+1:] + "\n")
        processed_images_count += 1
    else:
        with open(SKIPPED_IMAGES_FILE, "a") as myfile:
            myfile.write(image_path[subfolder_path_index+1:] + "\n")
        skipped_images_count += 1
    total_images_count = processed_images_count + skipped_images_count
    if (total_images_count) % 10 == 0:
        print("processed %d/%d images..." % (total_images_count, len(unprocessed_images)))
        
print "Analyzed %d images: %d used, %d skipped." % (processed_images_count + skipped_images_count, 
                                                   processed_images_count, skipped_images_count)

Number of previously processed images: 93
Number of images to process: 284


## Retraining

### Graph names settings

In [18]:
current_dir = os.getcwd()
PROCESSED_IMAGES_ABSOLUTE_PATH = os.path.join(current_dir, PROCESSED_IMAGES_FOLDER)
GRAPH_ABSOLUTE_PATH = os.path.join(current_dir, FOLDER_GRAPH)
RETRAINED_GRAPH_NAME = "inception_v3_retrained.pb"
OPTIMIZE_FOR_INFERENCE_GRAPH_NAME = "inception_v3_optimized.pb"
GRAPH_RETRAINED_ABSOLUTE_PATH = os.path.join(GRAPH_ABSOLUTE_PATH, RETRAINED_GRAPH_NAME)
GRAPH_OPTIMIZE_FOR_INTERFERENCE_ABSOLUTE_PATH = os.path.join(GRAPH_ABSOLUTE_PATH, OPTIMIZE_FOR_INFERENCE_GRAPH_NAME)
LABELS = os.path.join(GRAPH_ABSOLUTE_PATH, "labels.txt")

Run the following code to generate retraining command which can be pasted directly into console (TensorFlow root directory) to run retraining as described in [Inception retraining tutorial](https://www.tensorflow.org/versions/r0.9/how_tos/image_retraining/index.html)

In [20]:
print("bazel build -c opt --copt=-mavx tensorflow/examples/image_retraining:retrain")
print("""bazel-bin/tensorflow/examples/image_retraining/retrain \\\n --image_dir %s \\\n --model_dir %s \\\n --output_graph %s --output_labels %s""" % (PROCESSED_IMAGES_ABSOLUTE_PATH,
     GRAPH_ABSOLUTE_PATH, GRAPH_RETRAINED_ABSOLUTE_PATH, LABELS))

bazel build -c opt --copt=-mavx tensorflow/examples/image_retraining:retrain
bazel-bin/tensorflow/examples/image_retraining/retrain \
 --image_dir /Users/nikogamulin/workspace/tensorflow-retraining/images/processed \
 --model_dir /Users/nikogamulin/workspace/tensorflow-retraining/graph \
 --output_graph /Users/nikogamulin/workspace/tensorflow-retraining/graph/inception_v3_retrained.pb --output_labels /Users/nikogamulin/workspace/tensorflow-retraining/graph/labels.txt


### Optimize graph for inference

In [16]:
print("bazel build //tensorflow/python/tools:optimize_for_inference")
print("""bazel-bin/tensorflow/python/tools/optimize_for_inference \\\n --input %s \\\n --output %s --input_names=Mul \\\n --output_names=final_result""" % (GRAPH_RETRAINED_ABSOLUTE_PATH,
     GRAPH_OPTIMIZE_FOR_INTERFERENCE_ABSOLUTE_PATH))

bazel build //tensorflow/python/tools:optimize_for_inference
bazel-bin/tensorflow/python/tools/optimize_for_inference \
 --input /Users/nikogamulin/workspace/tensorflow-retraining/graph/inception_v3_retrained.pb \
 --output /Users/nikogamulin/workspace/tensorflow-retraining/graph/inception_v3_optimized.pb --input_names=Mul \
 --output_names=final_result


## Copy graph to Andoid example folder

In [22]:
ANDROID_EXAMPLE_FOLDER = "/Users/nikogamulin/workspace/tensorflow/tensorflow/examples/android/assets"

Paste the following commands to shell and run to copy retrained (optimized) graph to android folder

In [23]:
print("cp %s %s" %(GRAPH_OPTIMIZE_FOR_INTERFERENCE_ABSOLUTE_PATH, ANDROID_EXAMPLE_FOLDER))
print("cp %s %s" %(LABELS, ANDROID_EXAMPLE_FOLDER))

cp /Users/nikogamulin/workspace/tensorflow-retraining/graph/inception_v3_optimized.pb /Users/nikogamulin/workspace/tensorflow/tensorflow/examples/android/assets
cp /Users/nikogamulin/workspace/tensorflow-retraining/graph/labels.txt /Users/nikogamulin/workspace/tensorflow/tensorflow/examples/android/assets


### Run the following command to build .apk:

bazel build -c opt --android_cpu=armeabi-v7a tensorflow/examples/android:tensorflow_demo